# Dynamic topic modelling via Non-negative Matrix Factorization (NMF)

Warning: running the whole notebook requires more than 16Gb RAM. 
However you can run steps 1 and 2 separately

In [ ]:
! pip3 install gensim
! pip3 install razdel
! pip3 install pymorphy2

Import some preprocessing tools

In [1]:
import sys
sys.path.insert(0, '../../../preprocessing')
import preprocessing_tools_updt as preprocessing_tools

In [1]:
import pandas as pd;
import numpy as np;
import scipy as sp;
import sklearn;
import sys;
from nltk.corpus import stopwords;
import nltk;
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer;
from sklearn.decomposition import NMF;
from sklearn.preprocessing import normalize;
import re;
from io import StringIO;
from nltk.corpus import stopwords
import gc

# Step 1: Preprocessing

Here we will process input .csv line by line as far as the separator "|" happens to be a part of the message body for some entries

In [ ]:
date  = []
url   = []
host  = []
topic = []
title = []
body  = []
with open('../../../../../../project_news/interfax_sorted_2008-02-11_2019-01-19.csv') as news_file:
    for line in news_file:
        fields = line.split("|")
        date.append(fields[0])
        url.append(fields[1])
        host.append(fields[2])
        topic.append(fields[3])
        title.append(fields[4])
        body.append(" ".join(fields[5:]))
    news_file.close()
data_prep = pd.DataFrame({'date' : date,
                          'url'  : url,
                          'host' : host,
                          'topic': topic,
                          'title': title,
                          'body' : body},
                         columns={'date', 'url', 'host', 'topic', 'title', 'body'})
data_prep.info()

In [3]:
data = pd.read_csv('../../../../../../project_news/interfax_sorted_2008-02-11_2019-01-19.csv',
                   sep='|',
                   encoding='utf-8',
                   error_bad_lines=False);
data.columns = ["date", "url", "host", "topic", "title", "body"]
data.info()

b'Skipping line 100300: expected 6 fields, saw 14\n'
b'Skipping line 138281: expected 6 fields, saw 7\nSkipping line 139718: expected 6 fields, saw 7\nSkipping line 184612: expected 6 fields, saw 7\nSkipping line 184720: expected 6 fields, saw 7\nSkipping line 188784: expected 6 fields, saw 8\nSkipping line 197770: expected 6 fields, saw 12\nSkipping line 201959: expected 6 fields, saw 7\nSkipping line 226778: expected 6 fields, saw 7\n'
b'Skipping line 332321: expected 6 fields, saw 8\nSkipping line 388909: expected 6 fields, saw 7\n'
b'Skipping line 393299: expected 6 fields, saw 7\nSkipping line 394088: expected 6 fields, saw 7\nSkipping line 396155: expected 6 fields, saw 7\nSkipping line 397254: expected 6 fields, saw 7\nSkipping line 397433: expected 6 fields, saw 10\nSkipping line 397542: expected 6 fields, saw 12\nSkipping line 398285: expected 6 fields, saw 14\nSkipping line 398351: expected 6 fields, saw 7\nSkipping line 401967: expected 6 fields, saw 9\nSkipping line 403339:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 529025 entries, 0 to 529024
Data columns (total 6 columns):
date     529025 non-null object
url      529025 non-null object
host     529025 non-null object
topic    529025 non-null object
title    529025 non-null object
body     529025 non-null object
dtypes: object(6)
memory usage: 24.2+ MB


As we can see, 106 entries are lost due to extra separators. However, I assume, it is not critical.

In [ ]:
del data_prep
del date
del url
del host
del topic
del title
del body
gc.collect()

In [4]:
data["cleaned_text"] = data["body"].apply(preprocessing_tools.clean_text)
data[["cleaned_text"]].head()

,cleaned_text
0,барнаул февраля interfaxru первый городской ...
1,чита февраля interfaxru эксглава нк юкос мих...
2,москва февраля interfaxru в результате пожар...
3,токио февраля interfaxru рост ввп россии в ...
4,москва февраля interfaxru цены на нефть выро...


In [5]:
stopword_ru = stopwords.words('russian')
data["tokenized_text"] = data["cleaned_text"].apply(preprocessing_tools.lemmatization, stopword=stopword_ru)
data[["tokenized_text"]].head()

,tokenized_text
0,"[барнаул, февраль, interfaxru, городской, чемп..."
1,"[чита, февраль, interfaxru, эксглава, нк, юкос..."
2,"[москва, февраль, interfaxru, результат, пожар..."
3,"[токио, февраль, interfaxru, рост, ввп, россия..."
4,"[москва, февраль, interfaxru, цена, нефть, выр..."


NMF expects input texts as strings

In [6]:
tockens_clean = data["tokenized_text"].tolist()
texts_clean = [" ".join(x) for x in tockens_clean]

In [7]:
del data
gc.collect()

70

Save preprocessed texts into a file

In [8]:
with open('./clean.texts.txt', 'wt') as out_file:
    for doc in texts_clean:
        out_file.write(doc + '\n')
    out_file.close()

# Step 2: Topic modelling

Now we will read the preprocessed data

In [5]:
documents = []
with open('clean.texts.txt', 'rt') as in_file:
    for line in in_file:
        documents.append(line.rstrip())
    in_file.close()

In [6]:
documents[0]

'барнаул февраль interfaxru городской чемпионат ледолазание проходить воскресение барнаул надеяться проведение подобный чемпионат стать барнаул традиция сказать выступать церемония открытие чемпионат представитель федерация ледоскалолазание алтайский край выступить организатор соревнование сообщить корреспондент агентство интерфакссибирь соревнование проводиться среди мужчина женщина также вид парный гонка участник использование специальный снаряжение необходимый время забраться специально изготовить сосулька высота трехэтажный дом слово представитель организатор чемпионат олег язовский основной контингент участник любитель спортсмен иметь спортивный разряд'

In [7]:
max_features = 3000
vectorizer = CountVectorizer(analyzer='word', max_features=max_features);
x_counts = vectorizer.fit_transform(documents);
print(x_counts.shape)

(529025, 3000)


In [8]:
transformer = TfidfTransformer(smooth_idf=False);
x_tfidf = transformer.fit_transform(x_counts);

In [9]:
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)

Let us decompose the document-term matrix and experiment with the number of topics

200 topics

In [11]:
num_topics = 200
model = NMF(n_components=num_topics, init='nndsvd');
model.fit(xtfidf_norm)

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0,
  max_iter=200, n_components=200, random_state=None, shuffle=False,
  solver='cd', tol=0.0001, verbose=0)

In [14]:
# https://medium.com/mlreview/topic-modeling-with-scikit-learn-e80d33668730
def get_nmf_topics(model, n_top_words):
    
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [16]:
topics = get_nmf_topics(model, 20)
topics

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 90,Topic # 91,Topic # 92,Topic # 93,Topic # 94,Topic # 95,Topic # 96,Topic # 97,Topic # 98,Topic # 99
0,интерфакс,произойти,рубль,матч,дело,пожар,самолёт,индекс,город,сообщение,...,вертолёт,космический,осетия,ноябрь,январь,полиция,решение,чемпионат,декабрь,правительство
1,сообщить,инцидент,составить,стадион,уголовный,возгорание,борт,ммвб,житель,говориться,...,ми,аппарат,южный,москва,москва,полицейский,принять,мир,москва,премьерминистр
2,москва,причина,размер,игра,возбудить,пожарный,посадка,пункт,местный,отмечаться,...,крушение,спутник,абхазия,interfaxru,interfaxru,сотрудник,участие,футбол,interfaxru,вицепремьер
3,сказать,возгорание,триллион,болельщик,ук,ликвидировать,крушение,ртс,городской,поступить,...,посадка,орбита,грузинский,мх,новогодний,отдел,принимать,сборная,новогодний,заседание
4,среда,авария,сумма,тур,ст,тушение,пилот,фондовый,власть,заявление,...,борт,космодром,северный,месяц,австралия,местный,окончательный,европа,пермь,киргизия
5,корреспондент,предварительный,стоимость,против,отношение,огонь,ан,торг,администрация,распространить,...,пилот,запуск,независимость,конец,ск,столичный,принятие,проходить,ск,распоряжение
6,источник,пострадать,штраф,футбол,статья,лесной,разбиться,биржа,часть,прессрелиз,...,поиск,роскосмос,эдуард,марш,снег,стрельба,мера,фифа,пермский,кабинет
7,столичный,происшествие,доход,футбольный,факт,гореть,вылететь,вырасти,памятник,ведомство,...,спасатель,байконур,граница,состояться,домодедово,главка,связь,бразилия,прессконференция,премьер
8,interfaxru,обстоятельство,денежный,сыграть,расследование,кв,катастрофа,рынок,штат,разместить,...,разбиться,блок,признание,дата,минус,умвд,комиссия,хоккей,состояться,постановление
9,московский,данные,средство,го,возбуждение,возникнуть,совершить,фон,расположить,немой,...,катастрофа,мкс,конфликт,итоговый,открытый,доставить,процессуальный,открытый,конец,временной


While some topics are pretty clear:
Topic #1 - что-то связанное с передачей информации
Topic #2 - происшествия, несчастные случаи
Topic #3 - финансы, валюта
Topic #4 - футбол
Topic #5 - судебные дела
Topic #6 - пожары
Topic #7 - инциденты с авиационной техникой
...

Nevertheless, some topics overlap (like 7 and 90)

In [17]:
topics.to_csv('../nmf.topics.%d.csv' % num_topics, )

400 topics

In [18]:
num_topics = 400
model = NMF(n_components=num_topics, init='nndsvd');
model.fit(xtfidf_norm)

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0,
  max_iter=200, n_components=400, random_state=None, shuffle=False,
  solver='cd', tol=0.0001, verbose=0)

In [19]:
topics400 = get_nmf_topics(model, 20)
topics400

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 90,Topic # 91,Topic # 92,Topic # 93,Topic # 94,Topic # 95,Topic # 96,Topic # 97,Topic # 98,Topic # 99
0,интерфакс,произойти,миллиард,сборная,дело,пожар,самолёт,мск,линия,июль,...,спасатель,ноябрь,лондон,средство,январь,крым,решение,команда,продукция,председатель
1,сообщить,инцидент,составить,хоккей,расследование,тушение,борт,биржа,метрополитен,москва,...,отряд,москва,британский,денежный,москва,крымский,принять,сезон,россельхознадзор,заместитель
2,москва,обстоятельство,объесть,швеция,внутренний,пожарный,посадка,интерфаксюг,метро,interfaxru,...,спасательный,interfaxru,бибись,транспортный,interfaxru,севастополь,окончательный,формула,поставка,комитет
3,отметить,происшествие,чистый,финляндия,уголовный,ликвидировать,крушение,торг,столичный,лр,...,происшествие,мх,bbc,деньга,новогодний,полуостров,принятие,выступать,роспотребнадзор,правление
4,interfaxru,обрушение,прибыль,канада,фигурант,возникнуть,вылететь,расчёт,прямая,тариф,...,прибыть,итоговый,корпорация,хищение,австралия,электроэнергия,обжаловать,игрок,запрет,съезд
5,столичный,драка,резерв,женский,материал,гореть,ан,завтра,увеличить,нидерланды,...,гора,конец,передавать,кредитный,домодедово,порт,процессуальный,игра,ввоз,избранный
6,уточнить,причина,выручка,чехия,обвинять,склад,разбиться,начаться,упасть,официально,...,берег,марш,news,бюджетный,праздник,черноморский,соответствующий,национальный,онищенко,борис
7,пресссекретарить,устанавливаться,увеличиться,групповой,рассмотрение,причина,катастрофа,составить,восстановить,краснодарский,...,поиск,середина,ливия,база,транзит,автономный,отменить,играть,мясо,верховный
8,поступить,трагедия,триллион,швейцария,следствие,кв,лайнер,час,телефон,полугодие,...,река,месяц,лондонский,выделить,сбор,республика,руководство,молодёжный,предприятие,госдума
9,ранее,событие,внешний,норвегия,свидетель,сложность,авиакомпания,состояние,падение,попытка,...,эвакуировать,состояться,афганистан,использование,открытый,кавказ,решить,руководство,санитарный,кнр
